com_error: (-2147352567, '예외가 발생했습니다.', (0, 'Microsoft PowerPoint', "Slides.Item : Integer out of range. 1 is not in Index's valid range of 1 to 0.", '', 0, -2147188160), None)

In [9]:
import win32com.client

# 이미 실행 중인 PowerPoint 애플리케이션에 연결
try:
    ppt = win32com.client.GetActiveObject("PowerPoint.Application")
    print("기존 PowerPoint 애플리케이션에 연결됨")
except:
    # 실행 중인 PowerPoint가 없으면 새로 시작
    ppt = win32com.client.Dispatch("PowerPoint.Application")
    ppt.Visible = True
    print("새 PowerPoint 애플리케이션 시작됨")

# 현재 활성화된 프레젠테이션 가져오기
presentation = ppt.ActivePresentation
print(f"프레젠테이션 '{presentation.Name}' 접근됨")

# 현재 선택된 슬라이드 또는 특정 슬라이드에 접근
success_method = None  # 성공한 방법을 추적하는 변수
try:
    slide = ppt.ActiveWindow.View.Slide
    print(f"현재 활성 슬라이드 {slide.SlideIndex}번에 접근됨")
except:
    # 특정 슬라이드에 접근 (예: 첫 번째 슬라이드)
    slide_index = 1
    slide = presentation.Slides(slide_index)
    print(f"첫 번째 슬라이드에 접근됨")

# 노트 텍스트 설정하기 - 여러 방법 시도
notes_text = "dlalal"

# 방법 1: 노트 페이지에 직접 접근
try:
    slide.NotesPage.Shapes.Placeholders(2).TextFrame.TextRange.Text = notes_text
    print("방법 1 성공: NotesPage.Shapes.Placeholders(2) 사용")
    success_method = "방법 1: NotesPage.Shapes.Placeholders(2)"
except Exception as e:
    print(f"방법 1 실패: {str(e)}")
    
    # 방법 2: 다른 인덱스의 Shape 시도
    try:
        slide.NotesPage.Shapes(2).TextFrame.TextRange.Text = notes_text
        print("방법 2 성공: NotesPage.Shapes(2) 사용")
        success_method = "방법 2: NotesPage.Shapes(2)"
    except Exception as e:
        print(f"방법 2 실패: {str(e)}")
        
        # 방법 3: 모든 Shape 탐색
        try:
            shape_found = False
            for shape_idx in range(1, slide.NotesPage.Shapes.Count + 1):
                try:
                    shape = slide.NotesPage.Shapes(shape_idx)
                    if hasattr(shape, 'TextFrame') and hasattr(shape.TextFrame, 'TextRange'):
                        shape.TextFrame.TextRange.Text = notes_text
                        print(f"방법 3 성공: Shape {shape_idx}에 텍스트 설정됨")
                        success_method = f"방법 3: Shape {shape_idx}"
                        shape_found = True
                        break
                except Exception as inner_e:
                    print(f"  Shape {shape_idx} 처리 중 오류: {str(inner_e)}")
            
            if not shape_found:
                raise Exception("적합한 텍스트 프레임을 가진 Shape을 찾지 못함")
                
        except Exception as e:
            print(f"방법 3 실패: {str(e)}")
            
            # 방법 4: 노트 속성 직접 사용
            try:
                slide.Notes = notes_text
                print("방법 4 성공: slide.Notes 속성 사용")
                success_method = "방법 4: slide.Notes 속성"
            except Exception as e:
                print(f"방법 4 실패: {str(e)}")

# 결과 요약
if success_method:
    print("\n===== 결과 요약 =====")
    print(f"슬라이드 {slide.SlideIndex}번의 노트에 텍스트 설정 성공!")
    print(f"성공한 방법: {success_method}")
else:
    print("\n===== 결과 요약 =====")
    print("모든 방법 실패: 슬라이드 노트에 텍스트를 설정하지 못했습니다.")

print("\n작업 완료")

기존 PowerPoint 애플리케이션에 연결됨
프레젠테이션 'test_ppt_원본.pptx' 접근됨
현재 활성 슬라이드 3번에 접근됨
방법 1 성공: NotesPage.Shapes.Placeholders(2) 사용

===== 결과 요약 =====
슬라이드 3번의 노트에 텍스트 설정 성공!
성공한 방법: 방법 1: NotesPage.Shapes.Placeholders(2)

작업 완료


In [ ]:
import win32com.client # 예시: pywin32 사용 시
# 또는 from pptx import Presentation # 예시: python-pptx 사용 시 (API 약간 다름)

# --- Helper Functions (디버깅 추가) ---
def safe(obj, attr, default=None):
    """Safely get an attribute, returning default if an error occurs."""
    try:
        if obj is None:
            # print(f"!!! SAFE: Object is None when accessing '{attr}'") # Debug: Object None 확인
            return default
        if hasattr(obj, attr):
            val = getattr(obj, attr, default)
            # Check for common COM None/Error values if needed
            # Example: -2147467259 (0x80004005) might mean general failure
            if val is None or val == -1: # Check -1 specifically
                 # print(f"!!! SAFE: Got None or -1 for '{attr}'") # Debug: None/-1 값 확인
                 return default
            return val
        else:
            # print(f"!!! SAFE: Attribute '{attr}' not found in {type(obj)}") # Debug: 속성 부재 확인
            return default
    except Exception as e:
        # 중요: COM 오류 발생 시 어떤 속성에서 에러가 나는지 확인
        print(f"!!! SAFE function caught error accessing '{attr}' on {type(obj)}: {e}")
        # import traceback # 상세 스택 트레이스가 필요하면 주석 해제
        # traceback.print_exc()
        return default

def rgb_of(font):
    """Extracts RGB integer from font, handling various ways it might be stored."""
    if font is None:
        # print("  rgb_of: Font object is None") # Debug
        return None

    # print(f"  Checking font: {safe(font, 'Name')}") # Debug: Which font?
    rgb = None
    rgb1_raw = "N/A"
    rgb2_raw = "N/A"

    # 1. Try Fill.ForeColor
    try:
        fill = safe(font, "Fill")
        fore_color = safe(fill, "ForeColor")
        rgb1_raw = safe(fore_color, "RGB", "N/A") # Get raw value for debug
        # print(f"    Fill.ForeColor.RGB raw: {rgb1_raw}") # Debug print
        temp_rgb = safe(fore_color, "RGB", None)
        if temp_rgb is not None and temp_rgb != -1:
             rgb = temp_rgb
    except Exception as e:
        print(f"    Error getting Fill.ForeColor.RGB: {e}") # Debug

    # 2. If not found or invalid, try Color.RGB
    if rgb is None:
        try:
            color_obj = safe(font, "Color")
            rgb2_raw = safe(color_obj, "RGB", "N/A") # Get raw value for debug
            # print(f"    Color.RGB raw: {rgb2_raw}") # Debug print
            temp_rgb = safe(color_obj, "RGB", None)
            if temp_rgb is not None and temp_rgb != -1:
                 rgb = temp_rgb
        except Exception as e:
            print(f"    Error getting Color.RGB: {e}") # Debug

    result = rgb # Return the integer or None
    # print(f"    rgb_of using raw ({rgb1_raw}, {rgb2_raw}) -> result: {result}") # Debug print result
    return result


def snap(font):
    """
    Creates a comparable tuple representing key font attributes.
    NOW INCLUDES Strikethrough, Subscript, Superscript.
    """
    if font is None:
        return (None, 0.0, False, False, False, None, False, False, False) # Added defaults for new attrs

    size_val = safe(font, "Size", 0)
    try:
        size_float = float(size_val)
    except (ValueError, TypeError):
        size_float = 0.0

    # print(f"  Snapping font: {safe(font, 'Name')}, Size: {size_float:.1f}") # Debug: Basic font info
    snap_result = (
        safe(font, "Name"),
        round(size_float, 1),
        bool(safe(font, "Bold", 0)),
        bool(safe(font, "Italic", 0)),
        bool(safe(font, "Underline", 0)),
        rgb_of(font), # This will trigger its own debug prints
        # --- Added Attributes ---
        bool(safe(font, "Strikethrough", 0)),
        bool(safe(font, "Subscript", 0)),
        bool(safe(font, "Superscript", 0)),
    )
    # print(f"    Snap result: {snap_result}") # Debug: Print the full snap tuple
    return snap_result

def make_run_dict(text_range_segment):
    # (make_run_dict 함수는 이전과 동일하게 사용 가능, 필요시 safe 함수 등으로 보강)
    """Creates the run dictionary from a TextRange segment."""
    if text_range_segment is None:
        return {"Text": "", "Font": {}, "Hyperlink": None}

    f = safe(text_range_segment, "Font")
    text = safe(text_range_segment, "Text", "")

    # Handle potential None font object
    if f is None:
        font_dict = {
            "Name": None, "Size": None, "Bold": False,
            "Italic": False, "Underline": False,
            "Strikethrough": False, "Subscript": False, "Superscript": False # Add defaults
        }
    else:
        rgb = rgb_of(f) # rgb_of already handles None font internally now
        font_dict = {
            "Name": safe(f, "Name"),
            "Size": safe(f, "Size"),
            "Bold": bool(safe(f, "Bold", 0)),
            "Italic": bool(safe(f, "Italic", 0)),
            "Underline": bool(safe(f, "Underline", 0)),
            # Add new properties to the output dict as well
            "Strikethrough": bool(safe(f, "Strikethrough", 0)),
            "Subscript": bool(safe(f, "Subscript", 0)),
            "Superscript": bool(safe(f, "Superscript", 0)),
        }
        if rgb is not None:
            font_dict["Color"] = {
                "R": rgb & 0xFF,
                "G": (rgb >> 8) & 0xFF,
                "B": (rgb >> 16) & 0xFF,
            }

    # Hyperlink extraction (same as before)
    hyperlink_address = None
    try:
        action_settings = safe(text_range_segment, "ActionSettings")
        if action_settings:
            try:
                 # Index 1 corresponds to ppMouseClick
                 act = action_settings(1)
                 hl = safe(act, "Hyperlink")
                 hyperlink_address = safe(hl, "Address")
            except Exception:
                 pass # Ignore if action_settings(1) fails
    except Exception:
        pass

    return {
        "Text": text,
        "Font": font_dict,
        "Hyperlink": hyperlink_address,
    }


# --- Main Parsing Function (Revised Logic + Debug Prints) ---
def parse_text_frame_debug(text_frame):
    """
    Parse PowerPoint text frame with enhanced debugging for run splitting.
    Includes Strikethrough, Subscript, Superscript in comparison.
    Prints detailed snap comparison info.
    """
    out = {"Has Text": False}
    # ... (Initial checks for HasText, TextRange, full_text - same as previous revised version) ...
    if not safe(text_frame, "HasText", False):
        print("Debug: Frame has no text.")
        return out

    tr_all = safe(text_frame, "TextRange")
    if tr_all is None:
        print("Debug: Could not get TextRange object.")
        return out

    full_text = safe(tr_all, "Text", "")
    print(f"Debug: Full text length = {len(full_text)}")
    if not full_text:
        print("Debug: Text content is empty.")
        # Populate basic info even if empty (same as before)
        out.update({"Has Text": True, "Text": "", "Runs": [], "Paragraphs": {}, "Font": {}, "Hyperlink": None})
        # ... (Attempt to get base paragraph/font info - same as before) ...
        return out

    # Populate initial output structure (same as before)
    out.update({"Has Text": True, "Text": full_text, "Paragraphs": {}, "Font": {}, "Hyperlink": None})
    # ... (Populate base paragraph, font, hyperlink info - same as before) ...

    runs = []
    n = len(full_text)
    if n > 0:
        print("\n--- Starting Character Scan ---")
        try:
            cur_start_index = 1
            first_char_range = tr_all.Characters(cur_start_index, 1)
            print(f"Debug: Getting snap for Char {cur_start_index} ('{safe(first_char_range, 'Text', '?')}')")
            first_font = safe(first_char_range, "Font")
            cur_snap = snap(first_font)
            print(f"  Initial Snap: {cur_snap}")

            for i in range(2, n + 1):
                print(f"\nDebug: Checking Char {i}")
                char_range = tr_all.Characters(i, 1)
                char_text = safe(char_range, 'Text', '?')
                print(f"  Character: '{char_text}'")
                char_font = safe(char_range, "Font")

                if char_font is None:
                    print("  Warning: Font object is None for this character.")
                    # Decide how to handle: treat as different? same as last? For now, let's assume different
                    next_snap = (None,) * 9 # Create a distinctly different snap
                else:
                    next_snap = snap(char_font) # This will print internal details

                print(f"  Comparing: Current Snap = {cur_snap}")
                print(f"             Next Snap    = {next_snap}")

                if next_snap != cur_snap:
                    print(f"  >>> Snap changed! <<<")
                    segment_len = i - cur_start_index
                    print(f"    Creating Run: Start={cur_start_index}, Length={segment_len}")
                    segment_range = tr_all.Characters(cur_start_index, segment_len)
                    runs.append(make_run_dict(segment_range))

                    # Start the new run
                    cur_start_index = i
                    cur_snap = next_snap
                    print(f"    New Run Start: {cur_start_index}, New Snap: {cur_snap}")
                # else: # Optional: print if snap is the same
                #     print(f"  Snap is the same.")


            # Append the last run
            print(f"\nDebug: Appending final run.")
            last_segment_len = n - cur_start_index + 1
            print(f"  Final Run: Start={cur_start_index}, Length={last_segment_len}")
            last_segment_range = tr_all.Characters(cur_start_index, last_segment_len)
            runs.append(make_run_dict(last_segment_range))
            print("--- Character Scan Finished ---")

        except Exception as e:
            print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            print(f"Error during character-by-character run parsing: {e}")
            print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            import traceback
            traceback.print_exc() # Print detailed error stack
            # Fallback if scan fails badly
            if not runs:
                 print("Falling back to single run for the whole text due to error.")
                 runs.append(make_run_dict(tr_all))

    out["Runs"] = runs
    return out

# --- Example Usage ---
# (Same as before, but call parse_text_frame_debug instead)
# try:
#     # ... (get my_shape object) ...
#     if my_shape.HasTextFrame and my_shape.TextFrame.HasText:
#         text_frame = my_shape.TextFrame
#         parsed_data = parse_text_frame_debug(text_frame) # Call the debug version
        
#         print("\n--- Final Result ---")
#         print("Full Text:", parsed_data.get("Text"))
#         print("Number of Runs Found:", len(parsed_data.get("Runs", [])))
#         for i, run in enumerate(parsed_data.get("Runs", [])):
#             print(f"--- Run {i+1} ---")
#             # Limit text output for brevity if needed
#             run_text = run.get("Text", "")
#             print(f"  Text: '{run_text[:50]}{'...' if len(run_text) > 50 else ''}'") 
#             print("  Font:", run.get("Font"))
#             # print("  Hyperlink:", run.get("Hyperlink")) # Optional
#         # ... (print other info) ...

# except Exception as e:
#      print(f"An error occurred: {e}")

In [6]:
import win32com.client
import traceback  # 오류 추적을 위해 추가

# ============================================================================
# <<< 이전에 정의된 helper 함수들 (safe, rgb_of, snap, make_run_dict) 과
#     parse_text_frame_debug 함수를 그대로 가져와 사용하세요 >>>
# ============================================================================

def safe(obj, attr, default=None):
    """Safely get an attribute, returning default if an error occurs."""
    try:
        if obj is None:
            return default
        if hasattr(obj, attr):
            val = getattr(obj, attr, default)
            if val is None:
                return default
            return val
        return default
    except Exception:
        return default


def rgb_of(font):
    if font is None:
        return None
    rgb = None
    try:
        fill = safe(font, "Fill")
        fore = safe(fill, "ForeColor")
        temp = safe(fore, "RGB")
        if temp is not None:
            rgb = temp
    except Exception:
        pass
    if rgb is None:
        try:
            col = safe(font, "Color")
            temp = safe(col, "RGB")
            if temp is not None:
                rgb = temp
        except Exception:
            pass
    return rgb


def snap(font):
    if font is None:
        return (None, 0.0, False, False, False, None, False, False, False)
    size_val = safe(font, "Size", 0)
    try:
        size_f = float(size_val)
    except Exception:
        size_f = 0.0
    return (
        safe(font, "Name"),
        round(size_f, 1),
        bool(safe(font, "Bold", 0)),
        bool(safe(font, "Italic", 0)),
        bool(safe(font, "Underline", 0)),
        rgb_of(font),
        bool(safe(font, "Strikethrough", 0)),
        bool(safe(font, "Subscript", 0)),
        bool(safe(font, "Superscript", 0)),
    )


def make_run_dict(text_range_segment):
    if text_range_segment is None:
        return {"Text": "", "Font": {}, "Hyperlink": None}
    f = safe(text_range_segment, "Font")
    text = safe(text_range_segment, "Text", "")
    font_dict = {}
    if f:
        rgb = rgb_of(f)
        font_dict = {
            "Name": safe(f, "Name"),
            "Size": safe(f, "Size"),
            "Bold": bool(safe(f, "Bold", 0)),
            "Italic": bool(safe(f, "Italic", 0)),
            "Underline": bool(safe(f, "Underline", 0)),
            "Strikethrough": bool(safe(f, "Strikethrough", 0)),
            "Subscript": bool(safe(f, "Subscript", 0)),
            "Superscript": bool(safe(f, "Superscript", 0)),
        }
        if rgb is not None:
            try:
                font_dict["Color"] = {"R": rgb & 0xFF, "G": (rgb >> 8) & 0xFF, "B": (rgb >> 16) & 0xFF}
            except Exception:
                pass
    else:
        font_dict = {"Name": None, "Size": None, "Bold": False, "Italic": False,
                     "Underline": False, "Strikethrough": False, "Subscript": False,
                     "Superscript": False}

    hyperlink = None
    try:
        act = safe(safe(text_range_segment, "ActionSettings"), 1)
        hyperlink = safe(safe(act, "Hyperlink"), "Address")
    except Exception:
        pass

    return {"Text": text, "Font": font_dict, "Hyperlink": hyperlink}


def parse_text_frame_debug(text_frame):
    out = {"Has Text": False}
    if not safe(text_frame, "HasText", False):
        return out
    tr = safe(text_frame, "TextRange")
    if not tr:
        return out
    full = safe(tr, "Text", "")
    out.update({"Has Text": True, "Text": full, "Runs": []})
    if not full:
        return out
    runs = []
    n = len(full)
    try:
        cur_idx = 1
        cur_snap = snap(safe(tr.Characters(cur_idx, 1), "Font"))
        for i in range(2, n+1):
            nxt_snap = snap(safe(tr.Characters(i, 1), "Font"))
            if nxt_snap != cur_snap:
                seg_len = i - cur_idx
                if seg_len > 0:
                    runs.append(make_run_dict(tr.Characters(cur_idx, seg_len)))
                cur_idx = i
                cur_snap = nxt_snap
        last_len = n - cur_idx + 1
        if last_len > 0:
            runs.append(make_run_dict(tr.Characters(cur_idx, last_len)))
    except Exception as e:
        print(f"Error parsing runs: {e}")
        traceback.print_exc()
        runs.append(make_run_dict(tr))
    out["Runs"] = runs
    return out

In [ ]:
ppt_app = win32com.client.GetObject(Class="PowerPoint.Application")
presentation = ppt_app.ActivePresentation
slide = presentation.Slides(3)
print(f"분석 대상: 슬라이드 3 - '{safe(slide, 'Name', 'Unnamed')}', 도형 수: {slide.Shapes.Count}")

for j, shape in enumerate(slide.Shapes, start=1):
    try:
        # TextFrame 또는 TextFrame2 지원
        tf = None
        if safe(shape, "HasTextFrame", False):
            tf = shape.TextFrame
        elif safe(shape, "HasTextFrame2", False):
            tf = shape.TextFrame2
            # TextFrame2.TextRange는 TextFrame2.TextRange2 객체
        if tf is None or not safe(tf, "HasText", False):
            continue
        parsed = parse_text_frame_debug(tf)
        #print(parsed)
        runs = parsed.get("Runs", [])
        #print(f"도형 {j}: 발견된 Run 수: {len(runs)}")
        for r_i, run in enumerate(runs, start=1):
            txt = run.get("Text", "").strip()
            print(f"  Run {r_i}: '{txt}' Font={run.get('Font')}")
    except Exception as e:
        print(f"도형 {j} 처리 중 오류: {e}")


분석 대상: 슬라이드 3 - 'Slide3', 도형 수: 1
{'Has Text': True, 'Text': 'Multi-agent system 의 성능을 높이기 위해서 다양한 framework 들이 나왔지만, 이는 prompt 로 복잡하게 encoding 한 게 일반적이다. ⇒ contextual understanding ability 에 의해 저해됨.\r이런 것을 해결하기 위한 natural idea는 agent들을 finetuning하기 위한 막대한 collaborative data를 모으는 것. ⇒ 그러나 이는 모델의 general 능력을 없앨 수 있는 위험, AGI에 대한 열망에 모순됨.\r그래서 우리는 self-reflection mechanism 으로 collaboration proces를 optimize 하는 방법을 제안함.\rMulti-agent syste의 특징을 받아들이면서 agent-specific reflection을 위해 reflector prompt에 agent profile을 추가.', 'Runs': [{'Text': 'Multi-agent system 의 성능을 높이기 위해서 다양한 framework 들이 나왔지만, 이는 ', 'Font': {'Name': '새굴림', 'Size': 26.0, 'Bold': False, 'Italic': False, 'Underline': False, 'Strikethrough': False, 'Subscript': False, 'Superscript': False, 'Color': {'R': 0, 'G': 0, 'B': 0}}, 'Hyperlink': None}, {'Text': 'prompt 로 복잡하게 encoding ', 'Font': {'Name': '새굴림', 'Size': 26.0, 'Bold': True, 'Italic': False, 'Underline': False, 'Strikethrough': False, 'Subscript': False, 'Superscript': Fals

In [ ]:
Run 1: 'Multi-agent system 의 성능을 높이기 위해서 다양한 framework 들이 나왔지만, 이는' Font={'Name': '새굴림', 'Size': 26.0, 'Bold': False, 'Italic': False, 'Underline': False, 'Strikethrough': False, 'Subscript': False, 'Superscript': False, 'Color': {'R': 0, 'G': 0, 'B': 0}}
 

In [8]:
parsed

{'Has Text': True,
 'Text': '3',
 'Runs': [{'Text': '3',
   'Font': {'Name': 'Arial',
    'Size': 12.0,
    'Bold': False,
    'Italic': False,
    'Underline': False,
    'Strikethrough': False,
    'Subscript': False,
    'Superscript': False,
    'Color': {'R': 118, 'G': 118, 'B': 118}},
   'Hyperlink': None}]}

In [2]:
%pip install openai

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/661.2 kB ? eta -:--:--
   ---------------------------------------- 661.2/661.2 kB 5.1 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from test_Applier import test_json_Applier
import os
import json
from dotenv import load_dotenv

GEMINI_API_KEY = os.environ.get('GEMINI')

# Define the JSON as a Python dictionary directly
processed_json = {
    'understanding': 'Translate the content of slide 3 into English.',
    'tasks': [{
        'page number': 3,
        'description': 'Translate all text content on slide 3 to English.',
        'target': 'All text elements on slide 3 (title, body text, footnotes, headers, footers, chart/table labels, image captions, etc.)',
        'action': 'Translate to English using a high-quality machine translation API with post-editing option.',
        'contents': {
            'Presentation_Name': 'NLP_study_agent_KDJ.pptx',
            'Total_Slide_Number': '26',
            'Slide_Properties': {
                'Slide Layout Code': 16,
                'CustomLayout Name': '제목 및 내용',
                'CustomLayout Index': 2,
                'Background Fill Type': 'Solid',
                'Transition Effect': 0,
                'Advance Time (s)': 0.0,
                'Advance On Click': True,
                'Advance On Time': False
            },
            'Objects_Overview': 'Found 3 objects in slide number 3.',
            'Objects_Detail': [
                # Title object
                {
                    'Object_number': 1,
                    'Name': 'Title 1',
                    'Type': 'Placeholder',
                    'Position_Left': 66.0,
                    'Position_Top': 28.75,
                    'Size_Width': 828.0,
                    'Size_Height': 104.37503814697266,
                    'More_detail': {
                        'TextFrame': [
                            {
                                'RunIndex': 1,
                                'Text': 'Introduction',
                                'Font': {
                                    'Name': 'Arial',
                                    'Size': 44.0,
                                    'Bold': True,
                                    'Italic': False,
                                    'Underline': False,
                                    'Strikethrough': False,
                                    'Subscript': False,
                                    'Superscript': False,
                                    'Color': {'R': 0, 'G': 0, 'B': 0}
                                }
                            }
                        ]
                    }
                },
                # Content object
                {
                    'Object_number': 2,
                    'Name': 'Content Placeholder 2',
                    'Type': 'Placeholder',
                    'Position_Left': 66.0,
                    'Position_Top': 143.75,
                    'Size_Width': 828.0,
                    'Size_Height': 342.6250305175781,
                    'More_detail': {
                        'TextFrame': [
                            # The Korean text and formatting details
                            # Original text content segments would go here
                        ]
                    }
                },
                # Slide number
                {
                    'Object_number': 3,
                    'Name': 'Slide Number Placeholder 3',
                    'Type': 'Placeholder',
                    'Position_Left': 678.0,
                    'Position_Top': 500.5,
                    'Size_Width': 216.0,
                    'Size_Height': 28.75,
                    'More_detail': {
                        'TextFrame': [
                            {
                                'RunIndex': 1,
                                'Text': '3',
                                'Font': {
                                    'Name': 'Arial',
                                    'Size': 12.0,
                                    'Bold': False,
                                    'Italic': False,
                                    'Underline': False,
                                    'Strikethrough': False,
                                    'Subscript': False,
                                    'Superscript': False,
                                    'Color': {'R': 118, 'G': 118, 'B': 118}
                                }
                            }
                        ]
                    }
                }
            ],
            'Slide_Notes': {'Has Notes Page': False, 'Notes Content': None}
        },
        'content after edit': {
            # The content after translation would go here
            # This is the same structure but with translated text
        }
    }]
}

applier = test_json_Applier(model="gemini-2.5-flash-preview-04-17", api_key=GEMINI_API_KEY)
result = applier(processed_json)

start

Processing task 1:
{'page number': 3, 'description': 'Translate all text content on slide 3 to English.', 'target': 'All text elements on slide 3 (title, body text, footnotes, headers, footers, chart/table labels, image captions, etc.)', 'action': 'Translate to English using a high-quality machine translation API with post-editing option.', 'contents': {'Presentation_Name': 'NLP_study_agent_KDJ.pptx', 'Total_Slide_Number': '26', 'Slide_Properties': {'Slide Layout Code': 16, 'CustomLayout Name': '제목 및 내용', 'CustomLayout Index': 2, 'Background Fill Type': 'Solid', 'Transition Effect': 0, 'Advance Time (s)': 0.0, 'Advance On Click': True, 'Advance On Time': False}, 'Objects_Overview': 'Found 3 objects in slide number 3.', 'Objects_Detail': [{'Object_number': 1, 'Name': 'Title 1', 'Type': 'Placeholder', 'Position_Left': 66.0, 'Position_Top': 28.75, 'Size_Width': 828.0, 'Size_Height': 104.37503814697266, 'More_detail': {'TextFrame': [{'RunIndex': 1, 'Text': 'Introduction', 'Font': {'

In [ ]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types


def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-flash-preview-04-17"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""INSERT_INPUT_HERE"""),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(
            thinking_budget=14028,
        ),
        response_mime_type="text/plain",
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    generate()


In [2]:
error_occurred = False

import win32com.client

# Example JSON diffs loaded from your backend—not shown here.
before_json = {
    "slides": [
        {}, {}, {},  # slides 1,2 (empty for brevity, not used)
        {
            "shapes": [
                {
                    "Name": "Title 1",
                    "Text": "Quarterly Results",
                    "Font": {"Bold": True, "Color": [0, 0, 0], "Size": 44}
                },
                {
                    "Name": "Content Placeholder 2",
                    "Text": "Revenue increased by 12% year-over-year.",
                    "Font": {"Bold": False, "Color": [87, 87, 87], "Size": 28}
                }
            ],
            "notes": "Present data for Q1 and Q2 only."
        }
    ]
}

after_json = {
    "slides": [
        {}, {}, {},
        {
            "shapes": [
                {
                    "Name": "Title 1",
                    "Text": "FY24 Q1-Q2 Results",
                    "Font": {"Bold": True, "Color": [0, 0, 128], "Size": 44}
                },
                {
                    "Name": "Content Placeholder 2",
                    "Text": "Revenue grew 12%. Q2 margins improved by 5%.",
                    "Font": {"Bold": False, "Color": [64, 64, 64], "Size": 28}
                }
            ],
            "notes": "Include Q3 projections if possible."
        }
    ]
}

def to_bgr(color_triplet):
    # Expects color_triplet as [R, G, B]; returns as BGR_int (for PowerPoint .RGB)
    r, g, b = color_triplet
    return b << 16 | g << 8 | r

def find_shape_by_name_and_text(slide, name, text):
    """
    Finds a shape matching both name and text. Returns None if not found.
    """
    for shape in slide.Shapes:
        if shape.Name == name:
            if hasattr(shape, "TextFrame") and shape.TextFrame.HasText:
                current_text = shape.TextFrame.TextRange.Text
                if current_text.strip() == text.strip():
                    return shape
    return None

def get_shape_by_name(slide, name):
    """
    Returns the *first* shape with a given name.
    """
    for shape in slide.Shapes:
        if shape.Name == name:
            return shape
    return None

def update_shape_text_and_format(shape, after_text, after_font):
    """
    Updates the text and font formatting of a shape as needed.
    """
    text_frame = shape.TextFrame
    if text_frame.HasText:
        current_text = text_frame.TextRange.Text
        if current_text != after_text:
            text_frame.TextRange.Text = after_text

        # Set formatting on the whole text range
        text_range = text_frame.TextRange
        # Bold: PowerPoint expects -1 (True) or 0 (False)
        if "Bold" in after_font:
            text_range.Font.Bold = -1 if after_font["Bold"] else 0
        if "Italic" in after_font:
            text_range.Font.Italic = -1 if after_font["Italic"] else 0
        if "Color" in after_font:
            text_range.Font.Color.RGB = to_bgr(after_font["Color"])
        if "Size" in after_font:
            text_range.Font.Size = after_font["Size"]

def compare_font_style(f1, f2):
    """
    Returns True if font style objects differ
    """
    keys = ["Bold", "Italic", "Color", "Size"]
    for k in keys:
        if k in f1 and k in f2:
            if f1[k] != f2[k]:
                return True
        elif (k in f1) != (k in f2):
            return True
    return False

def set_slide_notes(slide, notes_text):
    """
    Updates the slide notes with the given text.
    """
    try:
        notes_shape = slide.NotesPage.Shapes.Placeholders(2)
        notes_shape.TextFrame.TextRange.Text = notes_text
    except Exception as e:
        print(f"Warning: Could not set notes for slide {slide.SlideIndex}: {e}")

def process_slide_transform(slide, before_slide_json, after_slide_json):
    # --- Update shapes ---
    before_shapes = before_slide_json.get("shapes", [])
    after_shapes = after_slide_json.get("shapes", [])

    for after_shape in after_shapes:
        shape_name = after_shape["Name"]
        after_text = after_shape["Text"]
        after_font = after_shape.get("Font", {})

        # Try to find this shape in before state
        before_shape = next(
            (item for item in before_shapes if item["Name"] == shape_name),
            None
        )

        # Determine if text or formatting has changed
        update_required = False
        if before_shape:
            # Compare text or font properties
            text_changed = before_shape["Text"] != after_text
            font_changed = compare_font_style(before_shape.get("Font", {}), after_font)
            if text_changed or font_changed:
                update_required = True
        else:
            # If not present in 'before', always update
            update_required = True

        if update_required:
            # Locate the shape by name on the actual slide, regardless of text
            shape = get_shape_by_name(slide, shape_name)
            if shape:
                update_shape_text_and_format(shape, after_text, after_font)
            else:
                print(
                    f"Warning: Could not find shape '{shape_name}' on slide {slide.SlideIndex}"
                )

    # --- Update slide notes if changed ---
    before_notes = before_slide_json.get("notes", "")
    after_notes = after_slide_json.get("notes", "")
    if before_notes != after_notes:
        set_slide_notes(slide, after_notes)

# === Main Execution ===
ppt_app = win32com.client.GetActiveObject("PowerPoint.Application")
active_presentation = ppt_app.ActivePresentation

try:
    slide_idx = 3  # 1-based index
    if active_presentation.Slides.Count < slide_idx:
        raise IndexError("Slide 3 does not exist in the active presentation.")

    slide = active_presentation.Slides(slide_idx)

    before_slide_json = before_json["slides"][slide_idx - 1]
    after_slide_json = after_json["slides"][slide_idx - 1]

    process_slide_transform(slide, before_slide_json, after_slide_json)

except Exception as e:
    print(f"Error updating slide 3: {e}")